<h1>Part 3: Agent Crowd Based Evacuation Behavior </h1>

<h3>In this part, we modeled the boids evacuation behaviour using cellular automata. </h3>

<h3>Conceptual Model: </h3>
The particle’s movement probability grid is modified by the static and dynamic fields.
The probability for each action is computed by:
pij = NMijDijSij
Dij is the modifier value from the dynamic field.
Sij is the modifier value from the static field.
N is a normalization factor to insure probabilities sum to 1.
Note: Formula for pij does not have to be a simple product.
Generalized as pij = N f(Mij, Dij, Sij).
f(…) may include exponentials for different behavior.

In [ ]:
import matplotlib
#matplotlib.use("qt4agg")
import pylab as PL
import random as RD
import numpy as NP
from numpy import linalg as LA
from scipy.spatial import distance as dist
from Agent import Agent
from math import sqrt
from IPython.display import clear_output
%matplotlib inline

matplotlib.rcParams['figure.figsize'] = [5, 5]

RD.seed()

populationSize = 64
noiseLevel = 5
avoidanceRadius = 50
flockRadius = 100
boardDimension = 1000

#strengths are proportion of new weight vs proportion of old weight (0-1)
avoidanceStrength = 0.8
approachStrength = 0.5
alignStrength = 0.3
obstacleStrength = 1
goalStrength = 1
totalStrength = 0.1

flag = [False]

<h3>Rules for Dynamic Field CA:</h3>
If a pedestrian leaves a cell (x, y) the dynamic floor field Dxy corresponding to this cell is increased by ΔDxy.
A virtual trace is left by the motion of pedestrians.
A certain amount of the field is distributed among the neighboring cells to model diffusion.
Diffusion is necessary because pedestrians do not necessarily follow exactly in the footsteps of others.
The field strength is reduced by a decay constant δ to model decay of the field.
Implies that the lifetime of the trace is finite.

In [ ]:
def init1():
    global time, agents, walls, goalPos

    time = 0

    scenario1()

Scenario 1: There is one exit for the agents to leave from, in the goal position (goalpos [500,100])

In [ ]:
def scenario1():
    global agents, walls, goalPos, goals
    agents = []
    for i in range(populationSize):
        row = i % 8
        col = i / 8
        newAgent = Agent(row*50+300, col*50+300, RD.gauss(0, noiseLevel), RD.gauss(0, noiseLevel), 0, 0)
        agents.append(newAgent)
        
    walls = []
    for i in range(60):
        newWall = [i*5+100, 100]
        walls.append(newWall)
    for i in range(61):
        newWall = [i*5+600, 100]
        walls.append(newWall)
    for i in range(160):
        newWall = [i*5+100, 900]
        walls.append(newWall)
    for i in range(160):
        newWall = [100, i*5+100]
        walls.append(newWall)
    for i in range(160):
        newWall = [900, i*5+100]
        walls.append(newWall)
        
    goals = 1
    goalPos = [500,100]

Draw the grid.

In [ ]:
def draw():
    PL.cla()
    PL.plot([wall[0] for wall in walls], [wall[1] for wall in walls], 'ro')
    x = [ag.posX for ag in agents]
    y = [ag.posY for ag in agents]
    PL.plot(x, y, 'bo')
    
    PL.axis('scaled')
    PL.axis([0, boardDimension, 0, boardDimension])
    PL.title('t = ' + str(time))
    PL.pause(0.01)

Step function updates positions of each boid every time step and changes their respective velocities. Once each boid agent has reached the target they will 'escape', which is the escape function and will be effectively removed. 

In [ ]:
def step():
    """Update positions of each agent each time step"""
    global time, agents, walls, goalPos, flag

    time += 1
    
    for ag in agents:
        colVect = ag.collisions(ag, avoidanceRadius, agents)
        avgLoc, avgVel = ag.getFlock(agents, flockRadius)
        alignVect = ag.align(avgVel, populationSize)
        apprVect = ag.approach(avgLoc)
        avoidVect = ag.avoidObstacles(walls, avoidanceRadius, flag)
        goalVect = ag.goal(goalPos)

        
        # Limit the max velocity 
        limit = 50
        if ag.oldVelX > limit:
            ag.oldVelX = sqrt(ag.oldVelX - limit) + 0.8 * limit
        if ag.oldVelY > limit:
            ag.oldVelY = sqrt(ag.oldVelY - limit) + 0.8 * limit
        
        # If there are multiple exits, agents should move to the closest one
        if goals == 2:
            d1 = dist.euclidean([ag.posX, ag.posY], goalPos)
            d2 = dist.euclidean([ag.posX, ag.posY], goalPos2)
            if d1 < d2:
                goalVect = ag.goal(goalPos)
            else:
                goalVect = ag.goal(goalPos2)
            escape(goalPos2)
        escape(goalPos)
        
        weightTot = avoidanceStrength + alignStrength + approachStrength + obstacleStrength + goalStrength
        weights = [avoidanceStrength / weightTot, alignStrength / weightTot, approachStrength / weightTot, obstacleStrength / weightTot, goalStrength / weightTot]
            
        ag.newVelX = (1 - totalStrength) * ag.oldVelX + totalStrength * (colVect[0] * weights[0] + alignVect[0] * weights[1] + apprVect[0] * weights[2] + avoidVect[0] * weights[3] + goalVect[0] * weights[4])       
        ag.newVelY = (1 - totalStrength) * ag.oldVelY + totalStrength * (colVect[1] * weights[0] + alignVect[1] * weights[1] + apprVect[1] * weights[2] + avoidVect[1] * weights[3] + goalVect[1] * weights[4])
        
        # Update positions using new velocities
        # Last number changes speed on screen
        ag.posX += ag.newVelX * 0.1
        ag.posY += ag.newVelY * 0.1
        
        # Wraps agents around when they leave the screen
        # And shifts new weights to old weight position
        for ag in agents:
            ag.posX = ag.posX % boardDimension
            ag.posY = ag.posY % boardDimension
            ag.oldVelX = ag.newVelX
            ag.oldVelY = ag.newVelY

The Escape method removes the agents when they leave successfully:

In [ ]:
def escape(goalPos):
    global agents
    for ag in agents:
        if ag.posX < 100 or ag.posX > 900 or ag.posY < 100 or ag.posY > 900:
            agents.remove(ag)
    

def normalize(x, y):
    mag = float(NP.sqrt(x*x + y*y))
    newX = x/mag
    newY = y/mag
    return [newX, newY]

<h3>Scenario 2:</h3> There are two exits for the agents to leave from
1. ) in the goal position (goalpos [500,100])
2. ) in the goal position 2 (goalpos2 [500,900])

In [ ]:
def scenario2():
    global agents, walls, goalPos, goalPos2, goals
    agents = []
    for i in range(populationSize):
        row = i % 8.0
        col = i / 8.0
        newAgent = Agent(row*50+300, col*50+300, RD.gauss(0, noiseLevel), RD.gauss(0, noiseLevel), 0, 0)
        agents.append(newAgent)
        
    walls = []
    for i in range(60):
        newWall = [i*5+100, 100]
        walls.append(newWall)
    for i in range(61):
        newWall = [i*5+600, 100]
        walls.append(newWall)
    for i in range(60):
        newWall = [i*5+100, 900]
        walls.append(newWall)
    for i in range(61):
        newWall = [i*5+600, 900]
        walls.append(newWall)
    for i in range(160):
        newWall = [100, i*5+100]
        walls.append(newWall)
    for i in range(160):
        newWall = [900, i*5+100]
        walls.append(newWall)
        
    goals = 2
    goalPos = [500, 100]
    goalPos2 = [500, 900]
    
def init2():
    global time, agents, walls, goalPos

    time = 0

    scenario2()


<h3>Run A Scenario</h3>
itit1() will run scenario 1. To run scenario 2, change to init2()

Notice that the time it takes each frame to load increases as the program runs, this is because the 'lag' is caused by all of the calculations that are being done in real time so when there are less agents, there are less calculations to do.

In [ ]:
init1()
while(True):
    clear_output(wait=True)
    draw()
    step()